In [3]:
from SALib.plotting.bar import plot as barplot
from SALib.analyze import sobol
from SALib.sample import saltelli
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [4]:
mac_excel_path = r"\\projectsportal.ghd.com@SSL\DavWWWRoot\sites\pp15_01\comdainwaternswrural\ProjectDocs\Wyangala Dam\L11 - Suspended doors make safe"

In [5]:
def evaluate_model(weights):
    options = np.array([1,2,3,4,5,6,7])
    mca = np.array([
    [7, 7, 7, 3, 2, 1, 7],
    [1, 2, 3, 6, 5, 7, 4],
    [4, 1, 2, 6, 4, 7, 5],
    [3, 7, 7, 3, 1, 7, 7],
    [7, 3, 2, 5, 5, 1, 7],
    [2, 1, 5, 7, 6, 4, 3],
    [1, 4, 5, 7, 6, 3, 2],
    ])

#     mca = pd.read_excel(mac_excel_path)
    weighted_mca =np.transpose(weights) * mca
    evaluation = -weighted_mca.sum(axis=0)
    rank = evaluation.argsort()
    rank_order = rank.argsort()
    return rank[0] + 1

In [6]:
labels = ["Operability","Maintenance","Reliability","OH&S - Suspended Load","OH&S - Additional Risks","Capex","Constructability"
]

In [7]:
problem = {
    'num_vars': 7,
    'names':labels,
    'bounds':[[0.05, 0.3],
        [0.05, 0.3],
        [0.05, 0.3],
        [0.05, 0.3],
        [0.05, 0.3],
        [0.05, 0.3],
        [0.05, 0.3],]
}

In [8]:
param_values = saltelli.sample(problem, 5000)
row_sums = np.sum(param_values, axis=1)
param_values = param_values/row_sums[:,np.newaxis]
param_values

array([[0.10125807, 0.07157329, 0.17334967, ..., 0.11586486, 0.26711586,
        0.05932243],
       [0.22786706, 0.06149051, 0.14892928, ..., 0.09954257, 0.2294863 ,
        0.05096547],
       [0.09225948, 0.15408063, 0.15794445, ..., 0.1055682 , 0.24337784,
        0.05405057],
       ...,
       [0.11508445, 0.08904815, 0.17931176, ..., 0.10100359, 0.1815036 ,
        0.27648297],
       [0.11123523, 0.08606976, 0.17331433, ..., 0.09762533, 0.16233654,
        0.31377874],
       [0.11666521, 0.09027128, 0.18177472, ..., 0.10239094, 0.17026104,
        0.28028064]])

In [9]:
Y = np.zeros(param_values.shape[0])

for i in range(param_values.shape[0]):
    Y[i] = evaluate_model(param_values[i,:])
Y.size

80000

In [10]:
Si = sobol.analyze(problem, Y)

In [11]:
S2_df = pd.DataFrame(Si['S2'], columns=labels)

print('First order interactions')
print(np.array2string(Si['S1'], precision=4, separator=', '))
print('\n')
print('Second order interactions')
S2_df.set_index(pd.Index(labels), inplace=True)
rounded_df = S2_df.round(decimals=3)
rounded_df = rounded_df.replace(np.nan, '', regex=True)
rounded_df

First order interactions
[ 9.8348e-03, -9.0241e-05,  5.8147e-02,  4.8286e-02,  2.0750e-03,
  4.4634e-02,  3.4910e-01]


Second order interactions


,Operability,Maintenance,Reliability,OH&S - Suspended Load,OH&S - Additional Risks,Capex,Constructability
Operability,,0.007,0.012,0.022,0.01,0.012,0.009
Maintenance,,,-0.003,-0,0.001,-0,0.006
Reliability,,,,0.024,0.015,0.005,0.034
OH&S - Suspended Load,,,,,0.003,0.017,0.109
OH&S - Additional Risks,,,,,,-0.001,0.012
Capex,,,,,,,0.026
Constructability,,,,,,,


In [12]:
Si_copy = Si
keys_to_drop = ['ST_conf', 'S2', 'S2_conf', 'S1_conf']
for key in keys_to_drop:
    Si_copy.pop(key, None)
Si_df = pd.DataFrame.from_dict(Si_copy)
Si_df['Criteria'] = labels
Si_df.set_index('Criteria', inplace=True)
Si_df

,S1,ST
Criteria,,
Operability,0.009835,0.121008
Maintenance,-0.000090,0.071752
Reliability,0.058147,0.238193
OH&S - Suspended Load,0.048286,0.355745
OH&S - Additional Risks,0.002075,0.118618
Capex,0.044634,0.206654
Constructability,0.349102,0.684362


In [13]:
fig = go.Figure(
    data=[
        go.Bar(name='First Order', x=labels, y=Si_df.S1),
        go.Bar(name='Total Order', x=labels, y=Si_df.ST)
    ],
)
# Change the bar mode
fig.update_layout(barmode='group', template="plotly_white", plot_bgcolor="#FFFFFF")
fig.show()

In [14]:
fig.write_image("Sensitivity of Criteria.png", scale = 10)

In [15]:
unique, counts = np.unique(Y, return_counts=True)

In [16]:
option_labels = ['Option '+str(x) for x in range(1,8)]

In [17]:
fig = go.Figure(data=[
    go.Bar(name='Frquency Variance', x=option_labels, y=counts/sum(counts)*100),
])
fig.update_layout(barmode='group', template="plotly_white",)
fig.show()

In [18]:
fig.write_image("Success frequency at varying weightings.png", scale = 10)